In [12]:
import torch
from ultralytics import YOLO
import os
import pydicom
import numpy as np
from PIL import Image
import random
import csv
import shutil

In [13]:
model = YOLO('yolo11n.pt')



In [14]:
TRAIN_ON_GPU = torch.cuda.is_available()
print(TRAIN_ON_GPU)

# if TRAIN_ON_GPU:
#     capsule_net = capsule_net.cuda()

True


In [15]:
def convert_dicom_to_image(dicom_path, output_folder, image_format="jpg"):
    """
    Konvertiert eine DICOM-Datei in ein Graustufenbild und speichert es unter der UID als Dateinamen.
    :param dicom_path: Pfad zur DICOM-Datei.
    :param output_folder: Zielpfad für das konvertierte Bild.
    :param image_format: Bildformat (z.B. "jpg" oder "png").
    """

    # Lese die DICOM-Datei
    dicom_data = pydicom.dcmread(dicom_path)
    
    # Extrahiere die UID der Datei
    uid = dicom_data.SOPInstanceUID
    
    # Erstelle den Zielpfad, basierend auf der UID
    output_path = os.path.join(output_folder, f"{uid}.{image_format.lower()}")
    
    # Überprüfe, ob die Datei bereits existiert
    if os.path.exists(output_path):
        print(f"Bild mit UID {uid} existiert bereits. Überspringe...")
        return
    
    # Extrahiere das Bild
    image = dicom_data.pixel_array
    
    # Normalisiere die Pixelwerte (Skalierung auf 0-255)
    image = ((image - np.min(image)) / (np.max(image) - np.min(image)) * 255).astype(np.uint8)
    
    # Konvertiere das Bild zu Graustufen, falls es RGB-Daten enthält
    if len(image.shape) == 3 and image.shape[2] == 3:  # Wenn das Bild RGB ist
        image = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140])  # Standard RGB zu Graustufen
        image = image.astype(np.uint8)
    
    # Wenn das Bild in Graustufen (2D) vorliegt, erweitere es auf 3 Kanäle (optional)
    if image.ndim == 2:  # Graustufenbild (2D)
        image_rgb = np.stack([image] * 3, axis=-1)  # [H, W] -> [H, W, 3]
    else:
        image_rgb = image
    
    # Speichere das Bild als JPG oder PNG
    img = Image.fromarray(image_rgb)
    img.save(output_path, format=image_format.upper())
    print(f"Bild mit UID {uid} wurde gespeichert.")
    


def process_dicom_folders(nodule_folder, non_nodule_folder, output_folder, image_format="jpg"):
    """
    Verarbeitet die DICOM-Dateien in den Ordnern 'nodule' und 'non-nodule' und speichert sie als Bilder.
    :param nodule_folder: Pfad zum Ordner mit DICOM-Dateien der 'nodule' Klasse.
    :param non_nodule_folder: Pfad zum Ordner mit DICOM-Dateien der 'non-nodule' Klasse.
    :param output_folder: Zielordner für die konvertierten Bilder.
    :param image_format: Bildformat (z.B. "jpg" oder "png").
    """
    # Liste der DICOM-Dateien im 'nodule'-Ordner
    nodule_files = [f for f in os.listdir(nodule_folder) if f.endswith(".dcm")]
    
    # Liste der DICOM-Dateien im 'non-nodule'-Ordner
    non_nodule_files = [f for f in os.listdir(non_nodule_folder) if f.endswith(".dcm")]
    
    # Bestimme die Anzahl der 'nodule' Dateien
    nodule_count = len(nodule_files)
    
    # Wähle zufällig die gleiche Anzahl von 'non-nodule' Dateien aus wie 'nodule' Dateien
    selected_non_nodule_files = random.sample(non_nodule_files, nodule_count)
    
    # Verarbeite alle 'nodule'-Dateien
    for dicom_file in nodule_files:
        dicom_path = os.path.join(nodule_folder, dicom_file)
        convert_dicom_to_image(dicom_path, output_folder, image_format)
    
    # Verarbeite die zufällig ausgewählten 'non-nodule'-Dateien
    for dicom_file in selected_non_nodule_files:
        dicom_path = os.path.join(non_nodule_folder, dicom_file)
        convert_dicom_to_image(dicom_path, output_folder, image_format)


In [16]:
def split_random_dataset(source_folder, train_folder, val_folder, test_folder, train_ratio=0.8, val_ratio=0.1):
    """
    Teilt einen Datensatz zufällig in Training, Validierung und Test auf.
    
    :param source_folder: Ordner mit allen Bildern.
    :param train_folder: Zielordner für die Trainingsbilder.
    :param val_folder: Zielordner für die Validierungsbilder.
    :param test_folder: Zielordner für die Testbilder.
    :param train_ratio: Prozentsatz der Daten, die für das Training verwendet werden.
    :param val_ratio: Prozentsatz der Daten, die für die Validierung verwendet werden.
    """
    # Erstelle Zielordner, falls sie noch nicht existieren
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    
    # Hol alle Dateien aus dem Source-Ordner
    all_files = os.listdir(source_folder)

    # Mische die Dateien zufällig
    random.shuffle(all_files)

    # Berechne die Anzahl für den Split
    total_files = len(all_files)
    train_size = int(total_files * train_ratio)
    val_size = int(total_files * val_ratio)
    test_size = total_files - train_size - val_size
    
    # Wähle zufällig die entsprechenden Dateien aus
    train_files = all_files[:train_size]
    val_files = all_files[train_size:train_size+val_size]
    test_files = all_files[train_size+val_size:]

    # Kopiere die Dateien in die entsprechenden Ordner
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), train_folder)

    for file in val_files:
        shutil.copy(os.path.join(source_folder, file), val_folder)

    for file in test_files:
        shutil.copy(os.path.join(source_folder, file), test_folder)

    print(f"Trainingsdaten: {len(train_files)} Bilder")
    print(f"Validierungsdaten: {len(val_files)} Bilder")
    print(f"Testdaten: {len(test_files)} Bilder")

# Beispielaufruf
source_folder = "D:/Datasets/JPGs"  # Ordner mit allen Bildern
train_folder = "D:/Datasets/train"
val_folder = "D:/Datasets/val"
test_folder = "D:/Datasets/testing"

# split_random_dataset(source_folder, train_folder, val_folder, test_folder, train_ratio=0.8, val_ratio=0.1)

Trainingsdaten: 42950 Bilder
Validierungsdaten: 5368 Bilder
Testdaten: 5370 Bilder


In [17]:
# Beispielaufruf
nodule_folder = "D:/Datasets/Daten/nodule"  # Pfad zum Ordner mit 'nodule'-DICOM-Dateien
non_nodule_folder = "D:/Datasets/Daten/non-nodule"  # Pfad zum Ordner mit 'non-nodule'-DICOM-Dateien
output_folder = "D:/Datasets/JPGs"  # Zielordner für die konvertierten Bilder
os.makedirs(output_folder, exist_ok=True)

# Verarbeite die DICOM-Dateien
process_dicom_folders(nodule_folder, non_nodule_folder, output_folder, image_format="JPEG")


In [18]:
def convert_to_yolo_format(bbox, image_width=512, image_height=512, class_id=0):
    """
    Konvertiert die Bounding Box-Koordinaten zu YOLO-Format (normalisiert).
    :param bbox: Bounding Box als (xmin, ymin, xmax, ymax).
    :param class_id: Die Klasse des Objekts.
    :param image_width: Breite des Bildes (fix 512).
    :param image_height: Höhe des Bildes (fix 512).
    :return: YOLO-kompatible Annotation als String.
    """
    # Berechne normalisierte Koordinaten
    x_center = (bbox[0] + bbox[2]) / 2 / image_width
    y_center = (bbox[1] + bbox[3]) / 2 / image_height
    width = (bbox[2] - bbox[0]) / image_width
    height = (bbox[3] - bbox[1]) / image_height
    
    # Erstelle den YOLO-Annotation String
    return f"{class_id} {x_center} {y_center} {width} {height}"

def save_yolo_annotation(txt_path, yolo_annotation):
    """
    Speichert die YOLO-Annotation in einer Textdatei.
    :param txt_path: Zielpfad der Textdatei.
    :param yolo_annotation: YOLO-kompatible Annotation.
    """
    with open(txt_path, 'w') as f:
        f.write(yolo_annotation)

def process_annotations(csv_file, output_folder, image_width=512, image_height=512):
    """
    Verarbeitet alle Annotationsdaten aus einer CSV-Datei und speichert sie als YOLO-Textdateien.
    :param csv_file: Pfad zur CSV-Datei mit Annotationsdaten.
    
    :param output_folder: Ordner, in dem die YOLO-Textdateien gespeichert werden.
    :param image_width: Breite des Bildes (fix 512).
    :param image_height: Höhe des Bildes (fix 512).
    """
    os.makedirs(output_folder, exist_ok=True)
    
    # Lese die CSV-Datei mit den Annotationen
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Header überspringen, falls vorhanden
        
        for row in reader:
            uid_annotation = row[0]
            
            # Hole die Bounding Box-Koordinaten
            xmin, ymin, xmax, ymax = map(float, row[2:6])
            
            # Konvertiere die Bounding Box-Koordinaten ins YOLO-Format
            yolo_annotation = convert_to_yolo_format((xmin, ymin, xmax, ymax), image_width, image_height)
            
            # Ziel-Textdateipfad (Textdatei hat denselben Namen wie die UID)
            txt_path = os.path.join(output_folder, f"{uid_annotation}.txt")
            
            # Speichere die Annotation
            save_yolo_annotation(txt_path, yolo_annotation)
            print(f"Annotation gespeichert für {uid_annotation} unter {txt_path}")

# Beispielaufruf
csv_file = "C:/Users/chris/Documents/GitHub/Masterarbeit_Dominik/Christoph/Object_Detection/annotation_files.csv"  # Pfad zur CSV-Datei mit den Annotationsdaten
 # Ordner mit den Bildern
output_folder = "D:/Datasets/TXTs"  # Ordner zum Speichern der YOLO-Textdateien

# Verarbeite die Annotationsdaten und speichere sie im YOLO-Format
process_annotations(csv_file, output_folder)


In [ ]:
import os
import shutil

# Pfade
base_dir = "D:/Datasets/YOLO_Dataset"  # Basisordner mit train, val, test
label_dir = "D:/Datasets/YOLO_Dataset/labels"  # Ordner, in dem sich die ursprünglichen TXT-Dateien befinden

splits = ['train', 'val', 'test']
for split in splits:
    #Bilder-Ordner-Pfad
    image_dir = os.path.join(base_dir, split, 'images')
    #New-Label-Folder
    split_label_dir = os.path.join(base_dir, split, 'labels')
    os.makedirs(split_label_dir, exist_ok=True)
    
    # Alle Bilder im aktuellen Split durchgehen
    for image_file in os.listdir(image_dir):
        if image_file.endswith(('.jpg', '.png', '.jpeg')):
            # Zugehörige Label-Datei ermitteln
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(label_dir, label_file)
            split_label_path = os.path.join(split_label_dir, label_file)
            
            # Verschieben der Label-Datei, falls vorhanden
            if os.path.exists(label_path):
                shutil.move(label_path, split_label_path)

print("Labels verschoben und leere Dateien erstellt.")

Labels verschoben und leere Dateien erstellt.


In [3]:
import os
import shutil

# Pfade zu den Dataset-Splits
base_dir = "D:/Datasets/YOLO_Dataset"  # Basisordner mit train, val, test
label_dir = "D:/Datasets/YOLO_Dataset/labels"  # Ordner mit den ursprünglichen TXT-Dateien

splits = ['train', 'val', 'test']
unmatched_labels = []  # Liste für nicht zugeordnete TXT-Dateien

# Alle TXT-Dateien durchlaufen
for txt_file in os.listdir(label_dir):
    if txt_file.endswith('.txt'):
        txt_name = os.path.splitext(txt_file)[0]  # Name ohne Erweiterung
        label_path = os.path.join(label_dir, txt_file)
        found = False

        # In den drei Splits nach dem passenden Bild suchen
        for split in splits:
            image_dir = os.path.join(base_dir, split, 'images')
            split_label_dir = os.path.join(base_dir, split, 'labels')
            os.makedirs(split_label_dir, exist_ok=True)

            # Prüfen, ob ein passendes Bild existiert
            for image_file in os.listdir(image_dir):
                if os.path.splitext(image_file)[0] == txt_name:
                    # Passendes Bild gefunden, Label kopieren
                    shutil.move(label_path, os.path.join(split_label_dir, txt_file))
                    found = True
                    break
            
            if found:
                break
        
        # Falls kein passendes Bild gefunden wurde
        if not found:
            unmatched_labels.append(txt_file)

# Ergebnisse anzeigen
print("Kopieren abgeschlossen.")
if unmatched_labels:
    print(f"Die folgenden Labels konnten nicht zugeordnet werden:\n{unmatched_labels}")
else:
    print("Alle Labels wurden erfolgreich kopiert.")


Kopieren abgeschlossen.
Die folgenden Labels konnten nicht zugeordnet werden:
['1.3.6.1.4.1.14519.5.2.1.6655.2359.100095183134113767852917005667.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100182597427431508452123585222.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100199112995438635843231609895.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100208251972139731117503233089.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100418984157618195673915127167.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100440817370820128908219881694.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100467178848702373061947594733.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100548455650970550282264501635.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100569778901449929574559846784.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100580691708822138145046680808.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100666625592252528839932188195.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.100703229327904293271883495281.txt', '1.3.6.1.4.1.14519.5.2.1.6655.2359.1007533866731427891359